In [1]:
import tensorflow as tf
from tensorflow import keras

# Resnet

In [ ]:
class Resnet:
  def __init__(self, images, labels, num_classes):
    self.num_classes = num_classes

  def x10(self):
    return self.num_classes * 10


In [ ]:
def Resnet18(num_classes):
  return Resnet(num_classes)

### Basic Block

In [20]:
class BasicBlock(keras.layers.Layer):

  def __init__(self, kernels, stride = 1):
    super().__init__()

    #----------------Features------------------#
    self.features = Sequential([
                                keras.layers.Conv2D(kernels, kernel_size=(3,3), strides=stride, padding="same"),
                                keras.layers.BatchNormalization(),
                                keras.layers.ReLU(),
                                keras.layers.Conv2D(kernels, kernel_size=(3,3), strides=1, padding="same"),
                                keras.layers.BatchNormalization()
    ])

    #-------------- Skip Connection------------#
    if stride != 1:
      skip = [
              keras.layers.Conv2D(kernels, (1, 1), strides=1),
              keras.layers.BatchNormalization()
      ]
    else:
      skip = []

    self.skip = Sequential(skip)
    # ------------------------------------------#

    def call(self, inputs, training = False):
      residual = self.skip(inputs, training = training)
      x = self.features(inputs, training = training)
      x = tf.nn.relu(keras.layers.add([residual, x]))
      return x

### Bottle Neck Block

In [21]:
class BottleNeckBlock(keras.layers.Layer):
  def __init__(self):
    super().__init__()